# Daily Predictions - UI

## Introduction

This notebook is a user interface for the daily predictions model. It allows the user to input the date for which they would like to see the predictions and then displays the predictions for that date.

### Usage

1. User Logins into the system
2. User is presented with a default list of instruments which are pretrained to predict the daily returns.
3. User selects the instrument for which they would like to see the predictions.
4. User is shown the prediction for the current Month, Week and Day.
5. User can add or delete an instrument to their watchlist.
6. Given the list of instruments, the user can see thumbnail views of all predictions.
   - Instrument name, Month, Week and Day predictions are shown.
7. Cross market analysis is also shown for the selected instrument.
   - User can see which other instruments are correlating and cross-correlating between the selected instrument and other instruments.
8. User can also see the predictions for the selected instrument for the current Month, Week and Day.
   - User can sort the list based on the strength of prediction values adding up Month/Week/Day predictions.

### Technologies

- StreamLit and StreamLit-Pydantic - For the User Interface
- StreamLit Lightweight Charts - For displaying the predictions
- PricePredictor Class - For making predictions
- statsmodels - For Time Series Analysis
- Screen Scraper - For fetching the data from other sites, if required. Financials, News etc.
- Async Threads - Use ncurrent.futures.ThreadPoolExecutor() for async operations
   - Fetching data, doing the predictions, and additional operations can be done in parallel.
   - Is it possible to update the session_state dataframe to highlight processed/unprocessed and in-progress items?  
     - This does not seem possible with the current streamlit.dataframe.

### Operations

   - Get list of most traded instruments
   - Generage predictions for the instruments as needed
   - List the instruments sorted by the strength of the predictions
     - Potential Components
       - Weekly Prediction
       - Day Prediction
       - Seasonal Prediction
   - Perform instrument cross correlation analysis


### Async Processing

   - Keeping async processes running in the background between st.rerun() calls, requires running the async process outside of the scope of the __main__ block.
   - Our background process needs to updates the st.progress() bar, within the scope of the __main__ block, which means that it will get destroyed when the st.rerun() is called in the __main__ block.
Upon an st.rerun() within the __main__ block, we need to check if the async process is running.
   - If it is running, we need to send it a new st.progress() bar to update.
   - Within the async process, we need to check if the st.progress() bar is still valid.
   - If it is not valid, we need to wait for a message from the __main__ block that contains a new st.progress() bar.
   - A simpler approach is to keep the st.progress() bar in the st.session_state object.
   - We can try this approach first, and if it fails, we can try the more complex approach.


### Needed: Async Progress Bar

   - The main problem is that the progress bar is not built for async operations.
      - To make the progress bar work with async operations, it needs to be updated
        such that it can call an that it passes itself to, the async function.
      - This means that the progress bar needs to be held in the session_state object
        such that it is not destroyed when the main thread is rerun, and the
        progress bar itself needs to be able to update itself in an async manner.

#### Function Signatures

   [Created GitHub issue #9310https](https://github.com/streamlit/streamlit/issues/9310)

   - async_progress(ss_name='async_prog_bar', async_func=async_func, af_args=[af_args], af_kwargs={af_kwargs})
       - The async_progress object upon instantiation should check for existing state in the session_state object and update itself accordingly.
       - Then it should spawn an async thread within the __main__ block that periodically updates the progress bar data in the session_state object.
       - The async thread should stop if it can no-longer find it's associated data in the session_state object.
       - The async thread should also enable a url (http://localhost:9504/async/<session_id>/<element_name>) to receive update requests from an async java script that is associated to the progress bar element on the web page, which it periodically updates.
   - async_func(ss_prog_bar: str, *args, **kwargs) 
       - async_func must be async ie. 'async def async_func()'
       - async_func modifies the progress bar by referencing and updating the ss_prog_bar element the session_state object.
       - if async_func is itself generates threads, then only the main thread should update the progress bar.
       - async_func should stop if it can no-longer find it's associated progress-bar data in the session_state object.

#### async_progress annotations

   - @st.async_progress
      - Indicates that the function is an async_progress function.
      - Validates and/or sets the function to be async.



### Gotta live with a blocking progress bar for now

   - Pre process symbols, prep & save the ML models, generate charts, etc.
      - Save Weekly bars, Weekly UP/Down for weekly correlations
      - Save Daily bars, Daily UP/Down for daily correlations
      - Save last 2 seasonality points, Daily & Weekly for correlation
      - Save last 2 prediction points, Daily & Weekly for correlation
      - Save latest yahoo ticker data for the instrument
   - Generate a symbols list as each symbol is processed.
   - Store data to a SqlLite3 database.
   